# Title: PyTorch를 이용한 합성곱 신경망 구현

 - `합성곱(convolution)` 연산

<img src="https://datadiving.dothome.co.kr/Deep%203_6.webp" width=500>
 
<img src='https://datadiving.dothome.co.kr/Deep%203_7.jpg' width=700>

 - `Convolution`의 기능
  - `Convolution`은 `filter (kernel)`을 통하여 특정 **특징(feature)의 유무 및 위치**를 식별하는데 특화된 연산입니다.
  - 이러한 이유로 이미지 분류 (image classification) 외에도 `객체 위치 식별 (object localization)`, `객체 탐지 (object detection)`, `이미지 분할 (image segmentation)`과 같은 많은 어플리케이션에서 CNN이 사용되었습니다.
 
 <img src="https://datadiving.dothome.co.kr/Deep%203_1.jpg" width=800>

- CNN에 기반한 이미지 분류 네트워크는 
  - 1) 이미지의 **특징을 추출**하는 **CNN** 파트와 (`Conv2D`, `MaxPooling2D` 이용)
    - **추출하는 특징을 점점 구체화(예: 테두리 -> 눈,코,귀 -> 얼굴)**하기 위하여 **Conv layer 중간 중간에 Pooling layer 추가**

        <img src = "https://datadiving.dothome.co.kr/Deep%203_9.jpeg" width=1000>

    - 일반적으로 특징이 점점 구체화 될수록 많은 수의 특징 추출, **filter의 갯수 증가**
    - 많은 수의 Conv layer를 사용하기 위하여 **zero padding** 사용
      - **Conv layer를 통과했을 때 이미지의 크기가 줄어드는 것 방지**

  - 2) **추출된 특징을 이용하여 classification을 수행**하는 **MLP** 파트로 나뉩니다.
    - CNN의 출력은 3차원이기 때문에 MLP에 넣어줄 때 **1차원으로 reshape** (`Flatten` 이용) 해주어야 합니다.

<img src="https://datadiving.dothome.co.kr/Deep%203_4.png" width=1100> 

 - MLP에서와 마찬가지로 `conv layer`의 수와 `filter (feature map)`의 수는 **실험적**으로 결정되기 때문에 여러 개의 모델을 생성 및 학습하여 성능을 비교해보셔야 합니다.

 - 다음 세 가지 값을 입력으로 받아 그에 해당하는 합성곱 신경망을 생성해주는 함수를 만들어봅시다:
   - 입력 데이터 모양 `input_shape`
   - 결과 데이터 차원 `output_dim`
   - `conv layer`의 `filter` 수를 모아놓은 `num_filters_list` (예: num_filters_list = [16, 'max_pool', 32, 'max_pool', 64, 'max_pool', 128, 'max_pool'])
    - 'max_pool'인 경우 `pooling`을 적용하는 것으로 생각해봅시다.

### Step 0) PyTorch 패키지 import

 - 참고: **Tensorflow** 패키지

  - `tensorflow.keras.layers (layers)`: 딥러닝 네트워크를 설계할 때 층(layer) 관련 함수들(예:`Dense`, `Conv2D`, `MaxPooling2D`, `SimpleRNN`, `LSTM`)을 모아놓은 라이브러리

  - `tensorflow.keras.models.Model (Model)`: 생성한 층들을 연합하여 하나의 모델로 구성할 때 사용하는 함수

  - `tensorflow.keras.datasets`: TensorFlow에서 딥러닝 실습을 위해 제공해주는 데이터셋 (예: `mnist`, `cifar10`, `cifar100`, `imdb`)

In [ ]:
import torch

## 데이터 전처리 관련 패키지
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 딥러닝 모델 생성 관련 패키지
import torch.nn as nn

## 딥러닝 모델 업데이트 알고리즘 관련 패키지
import torch.optim as optim

## 다차원 데이터 처리를 위한 라이브러리
import numpy as np

## 데이터 시각화를 위한 라이브러리
import matplotlib.pyplot as plt

0) 데이터 전처리 관련 패키지

 - `torchvision.datasets`: 다양한 dataset들을 포함하고 있는 패키지 (https://pytorch.org/vision/stable/datasets.html)

 - `torchvision.transforms`: 데이터 변형을 위한 라이브러리

 - `torch.utils.data.DataLoader`: 데이터를 `batch size` 크기로 분할해주는 라이브러리


1) 딥러닝 모델 생성 관련 패키지

  - `torch.nn (nn)`: 딥러닝 모델을 생성할 때 층(layer) 관련 함수들(예: Linear, Conv2d, MaxPool2d, ReLU, BatchNorm2d)을 모아놓은 패키지

  - `torch.nn.Module (Module)`: 생성한 층들을 연합하여 하나의 모델로 구성할 때 사용하는 클래스

2) 딥러닝 모델 학습 관련 패키지

  - `torch.optim (optim)`: 모델 업데이트 알고리즘(`optimizer`)들을 모아놓은 패키지 

 ### Step 1) 데이터 불러오기 및 전처리 

 - CIFAR10 데이터셋 불러오기

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
     )

train_dataset = datasets.CIFAR10(root="../data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="../data", train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


 - **Data Augmentation (데이터 증강)**
  
  - 원본 데이터를 변형하여 데이터의 개수를 늘리는 기법 (**데이터 다양화**)
      - 변형의 예) 회전(rotation), 반전 or 대칭(flip), 잘라내기(crop)

      <img src="https://miro.medium.com/max/1400/0*LR1ZQucYW96prDte" width=700>

      - 참고) `ResNet` 학습에도 `Data Augmentation` 기법이 사용되었습니다! (https://arxiv.org/pdf/1512.03385.pdf)

  - `Data Augmenetation`을 사용하면 성능을 향상시킬 수 있을까?

    <img src='https://datadiving.dothome.co.kr/Deep%204_data%20augmentation.jpg' border='0'></a>

    **Flip**

    - 왼쪽만 바라보는 고양이가 train 때 주어졌다면 오른쪽을 보는 고양이는 test 때 맞추지 못할 수 있습니다.
    
    - 이 경우, 왼쪽만 바라보는 고양이를 좌우 대칭시켜 모두 학습에 사용하면 test 때 고양이가 어느 쪽을 보더라도 맞출 수 있게 됩니다.

    <img src='https://datadiving.dothome.co.kr/Deep%204_data%20augmentation%202.jpg' border='0'></a>

    **Crop**

    - 딥러닝 모델이 꼬리와 귀, 수염, 눈이라는 특징으로 고양이인지를 판단한다면 고양이가 침대 밑에 숨어있어 꼬리만 보이는 경우 고양이라고 판단하지 못할 수 있습니다.

    - 이 경우, 고양이의 각 부분을 잘 알려주기 위해 꼬리 부분만 잘라서 넣어준다면 성능이 더 좋아질 수 있을 것입니다.

    <img src='https://datadiving.dothome.co.kr/Deep%204_%EB%B0%9D%EA%B8%B0%20%EC%A1%B0%EC%A0%88.jpg' border='0'></a>

    **💡 밝기 조절**  
    - 만약 딥러닝 모델로 앱을 만들게 되면, 사진을 찍는 사람마다 빛의 양이 다르겠죠? 딥러닝이 동일하게 인식할 수 있도록 어두운 것부터 밝은 것까지 모든 사진의 밝기를 조절해서 넣어줍시다!    

 - PyTorch에서는 `torchvision.transforms` 라이브러리를 통하여 `data augmentation`을 쉽게 구현할 수 있습니다!

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        transforms.RandomCrop(size=32, padding=4),
        transforms.RandomHorizontalFlip()
    ]
)
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

Files already downloaded and verified


 - 뿐만 아니라, `DataLoader` 함수를 통하여 전체 데이터셋을 `배치 (batch)` 단위로 쪼갤 수 있습니다.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Step 2) 인공신경망 생성

 - PyTorch에서는 `torch.nn` 라이브러리 내의 함수들과 `클래스 (class)` 용법을 사용하여 인공신경망을 생성합니다.

    - **nn.Conv2d**: 합성곱 계층을 생성하는 함수

    - **nn.MaxPool2d**: 풀링 계층을 생성하는 함수

    - **nn.Flatten**: 다차원 데이터를 1차원으로 평탄화 시켜주는 함수

    - **nn.Linear**: 전결합계층을 생성하는 함수

In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding="same")
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="same")
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding="same")
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(in_features=2048, out_features=10)

    def forward(self, x):
        conv1 = nn.ReLU()(self.conv1(x))
        pool1 = self.pool1(conv1)

        conv2 = nn.ReLU()(self.conv2(pool1))
        pool2 = self.pool2(conv2)

        conv3 = nn.ReLU()(self.conv3(pool2))
        pool3 = self.pool3(conv3)

        flatten = self.flatten(pool3)

        logits = self.fc1(flatten)

        return logits

 - 의도에 맞게 네트워크가 생성되었는지 확인해봅시다!

    - **summary** 함수 이용

In [ ]:
from torchsummary import summary

model = MyModel()
model.to("cuda")
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
         MaxPool2d-2           [-1, 32, 16, 16]               0
            Conv2d-3           [-1, 64, 16, 16]          18,496
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5            [-1, 128, 8, 8]          73,856
         MaxPool2d-6            [-1, 128, 4, 4]               0
           Flatten-7                 [-1, 2048]               0
            Linear-8                   [-1, 10]          20,490
Total params: 113,738
Trainable params: 113,738
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.56
Params size (MB): 0.43
Estimated Total Size (MB): 1.01
----------------------------------------------------------------


### Step 3) 학습 (Training)
 
 - 손실 함수 (Loss) 및 업데이트 알고리즘 (Optimizer) 설정
    
    - 손실함수로는 **교차엔트로피오차 (cross entropy)**를
    
    - 업데이트 알고리즘으로는 **adam** optimizer를 사용하겠습니다.

        - `torch.optim (optim)`: 모델 업데이트 알고리즘(`optimizer`)들을 모아놓은 패키지 



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

 - PyTorch에서는 GPU를 사용할 경우, **모델과 데이터를 모두 GPU에 올려주어야 합니다.**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

MyModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2048, out_features=10, bias=True)
)

 - 👨🏻‍🏫 학습 순서

    1. 입력 (x)에 대한 결과 예측 -> y_pred

    2. y_pred와 정답 (y_true)을 비교하여 손실함수의 값 계산 -> **criterion(y_pred, y_true)**

    3. 손실함수의 값이 작아지는 방향으로 네트워크 업데이트 -> **optimizer.step()**

In [ ]:
epochs = 10

for epoch in range(epochs):  
    running_loss = 0
    
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        
        # 입력에 대한 결과 예측
        y_pred = model(x_batch)

        # 예측값과 정답을 비교하여 손실함수의 값 계산
        loss = criterion(y_pred, y_batch)

        # 손실함수의 값이 작아지는 방향으로 네트워크 업데이트
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in train_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch: {epoch + 1}, Train Loss: {running_loss / total}, Train Accuracy: {100 * correct / total}')

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}')

Epoch: 1, Train Loss: 0.04507628124833107, Train Accuracy: 57.25
Test Accuracy: 60.36
Epoch: 2, Train Loss: 0.03320949960827827, Train Accuracy: 66.852
Test Accuracy: 69.42
Epoch: 3, Train Loss: 0.028798986775279044, Train Accuracy: 69.17
Test Accuracy: 71.48
Epoch: 4, Train Loss: 0.02636519924044609, Train Accuracy: 72.058
Test Accuracy: 73.15
Epoch: 5, Train Loss: 0.0246919292140007, Train Accuracy: 74.768
Test Accuracy: 75.06
Epoch: 6, Train Loss: 0.023364896874427797, Train Accuracy: 75.782
Test Accuracy: 75.76
Epoch: 7, Train Loss: 0.022320393640995025, Train Accuracy: 76.148
Test Accuracy: 77.32
Epoch: 8, Train Loss: 0.02160057497948408, Train Accuracy: 77.264
Test Accuracy: 77.53
Epoch: 9, Train Loss: 0.02088353258550167, Train Accuracy: 78.264
Test Accuracy: 78.16
Epoch: 10, Train Loss: 0.02063079702347517, Train Accuracy: 77.912
Test Accuracy: 77.84


# Part 2) **배치 정규화(Batch Normalization)**

 - `Gradient Vanishing Problem`: `Backpropagation`에 의해 gradient가 앞단으로 전파되면서 점점 옅어지게 되어 너무 작아져서 **소멸**하게 되는 문제

    <img src="https://t1.daumcdn.net/cfile/tistory/997E1B4C5BB6EAF239" width=700>

    - Layer를 통과할 때마다 `활성함수의 미분값(gradient)`이 계속 곱해지는데, 기울기가 완만한 영역의 입력값이 주어지는 경우 앞단의 `gradient`가 아주 작아 모델이 업데이트 되지 않는 문제

 - `Internal Covariate Shift`: Layer를 통과할 때마다 입력값의 분포가 변화하는 현상

    <img src='https://datadiving.dothome.co.kr/Deep%204_internal%20Covariate%20Shift%202.jpg' width=700>

    - Layer가 깊어질수록 이전에 더 많은 layer를 통과하기 때문에 입력값의 분포가 많이 변하게 된다.

    <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/3.png" width=700>

    - 입력값의 분포가 점점 변형되어 `gradient`가 작은 영역에 도달하게 된다면 `vanishing gradient`문제가 발생한다.


  <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png" width=700>


 - `배치 정규화 (Batch Normalization)`: 각 레이어를 통과할 때마다 **정규화 (normalization)**하는 레이어를 두어 입력값의 분포가 변형되지 않도록 조절하는 알고리즘

  - 배치 단위로 실시하기 때문에 `batch normalization`이라고 부릅니다.

  - 활성함수의 입력값의 분포를 조절해주는 것이 목적이므로 **활성함수** 앞에 넣어준다.

  - 알고리즘 테이블

    <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcFYkLE%2FbtqEcUnlXKy%2FZbGZNjObjo2gL2xss8zYzk%2Fimg.png" width=500>

  - $\gamma, \beta$로 `scale and shift`를 해주는 이유?

  <img src="https://miro.medium.com/max/1400/1*0yhJ7DbhOX-tRUseljjYoA.png" width=700>

 - **VGG16**에 `BatchNormalization`을 추가한 모델을 PyTorch를 이용하여 생성해봅시다.

  <img src="https://miro.medium.com/max/857/1*AqqArOvacibWqeulyP_-8Q.png" width=700>

In [ ]:
def MyModel(input_shape, output_dim, num_filters_list, use_batch_norm):
  # 입력계층 (Input Layer)
  img = layers.Input(shape=input_shape) # cifar10의 경우는 input_shape=[32, 32, 3]

  # 특징 추출 파트 - CNN
  h = img
  for num_filters in num_filters_list:
    if num_filters == "max_pool": 
      h = layers.MaxPooling2D(pool_size=(2, 2))(h)
    else:
      h = layers.Conv2D(filters=num_filters, kernel_size=3, strides=1, padding="same")(h) # convolution 
      if use_batch_norm == True:
        h = layers.BatchNormalization()(h) # batch normalization
      h = layers.ReLU()(h) # activation

  # 분류 파트 - MLP
  mlp_input = layers.Flatten()(h)
  prob = layers.Dense(units=output_dim, activation="softmax")(mlp_input)

  # 전체 모델
  return Model(inputs=img, outputs=prob)

# Part 3) **ResNet** 구현

 - ResNet에서의 key idea는 2개의 layer를 통과할 때마다 `skip connection`을 넣어주어`backpropagation` 과정에서 **gradient가 잘 흐르도록 하는 것**입니다.
 
 <img src="https://images.velog.io/images/junyoung9696/post/3137e50c-b52f-4cdd-8ae8-2faf497efe84/r10.png" width=500>

 - ResNet 논문(https://arxiv.org/pdf/1512.03385.pdf)을 참고하여 CIFAR10 이미지 분류 네트워크를 생성해봅시다.

  <img src="https://blog.kakaocdn.net/dn/bQfaUX/btqYAtD1KcX/Zdc4DLFzR9SoJYBlO6M1uK/img.png" width=700>

 ### Step 1) Residual Block 구성